In [1]:
import csv
import os
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
from app.flipside.models import Flipside
from app.constants import oracle_feeds_lookup, oracle_feeds, phase_windows, liquidators, attackers, targets, timestamp_windows

In [3]:
filenames = {
    # "swaps": "ez_dex_swaps_filtered_participants" ,
    "swaps": "ez_dex_swaps_filtered_crv" 

}

In [ ]:
fs = Flipside(filenames)

In [5]:
window_start = phase_windows[1][0] - 420 # 16025000
window_end   = phase_windows[4][1] +420

def apply_window(df, window_start=window_start, window_end=window_end):
    return df.loc[
        (df["BLOCK_NUMBER"] > window_start) & 
        (df["BLOCK_NUMBER"] < window_end)
    ]

In [6]:
def chart_it(title, df, x, y, color, facet_col="SYMBOL"):
    fig = px.scatter(df, 
                     x=x,
                     y=y,
                     color=color,
                    title=title,
                    facet_col=facet_col,
)
    fig.update_layout(autotypenumbers='convert types')
    fig.show()

def fig_append_phase_labels(fig, start = None, end=None):
    for window in phase_windows:
        if window[0]>= start and window[1]<= end and window[3] != "":
            fig.add_vrect(x0=window[0], x1=window[1], 
                        annotation_text=window[2], annotation_position="top left",
                        fillcolor=window[3], opacity=0.25, line_width=0)
    return fig

# SWAPS

In [ ]:
df = fs.dataframes['swaps']
print(len(df.ORIGIN_FROM_ADDRESS.unique()))
print(len(df.ORIGIN_TO_ADDRESS.unique()))
df.head(10)
len(df)

In [8]:
# df = apply_window(df)
df_filtered = df[df.ORIGIN_TO_ADDRESS.isin(liquidators)]

In [ ]:
df_filtered.ORIGIN_FROM_ADDRESS.unique()
df.keys()

In [ ]:
fig = px.scatter(df, 
                 x=df.BLOCK_TIMESTAMP,
                 y=df.AMOUNT_IN_USD,
                 color="ORIGIN_TO_ADDRESS",
                title="CRV Swaps by market (USD)", 
                facet_col="PLATFORM",
                facet_col_wrap=3,
                height=700,
                )
fig.update_layout(autotypenumbers='convert types')
# fig = fig_append_phase_labels(fig, window_start, window_end)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liquidations', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)
fig.show()

In [ ]:
ts = df.BLOCK_TIMESTAMP[1]
day = ts[0:ts.find(" ")]
hour = ts[ts.find(" "): ts.find(":")]
aggregate_day = f"{day}{hour}:00:00.000"
print(aggregate_day)
print(ts)

In [12]:
def process_volume(df, is_hourly=True):

    block_records = {
        'timestamp': [], 
        'buy_volume': [], 
        'sell_volume': [],
        'total_volume': [],
        'buy_volume_usd': [], 
        'sell_volume_usd': [],
        'total_volume_usd': []
        }
    last_day = ""
    last_aggregate = None
    for index, row in df.iterrows():

        ts = row['BLOCK_TIMESTAMP']
        day = ts[0:ts.find(" ")]
        hour = ts[ts.find(" "): ts.find(":")]
        if is_hourly:
            aggregate_day = f"{day}{hour}:59:00.000"
        else:
            aggregate_day = f"{day} 23:59:59.000"


        is_same= False
        if day == last_day:
            if is_hourly:
                if hour == last_hour:
                    is_same = True
                    last_aggregate = aggregate_day
            else:
                is_same = True
                last_aggregate = aggregate_day              
                # print(last_aggregate)


        if not is_same:
            if last_aggregate == None:
                last_aggregate = aggregate_day
            else:
                block_records['timestamp'].append(last_aggregate)
                block_records['buy_volume'].append(buy_volume)
                block_records['sell_volume'].append(sell_volume)
                block_records['total_volume'].append(buy_volume + sell_volume)
                block_records['buy_volume_usd'].append(buy_volume_usd)
                block_records['sell_volume_usd'].append(sell_volume_usd)
                block_records['total_volume_usd'].append(buy_volume_usd + sell_volume_usd)

            last_aggregate = aggregate_day
            buy_volume = 0
            sell_volume = 0
            buy_volume_usd = 0
            sell_volume_usd = 0

            last_day = day
            last_hour = hour

        if row['SYMBOL_IN'] == 'CRV':
            buy_volume += row['AMOUNT_IN']
            buy_volume_usd += row['AMOUNT_IN_USD']
        else:
            sell_volume += row['AMOUNT_IN']
            sell_volume_usd += row['AMOUNT_IN_USD']          
    return block_records

In [13]:
block_records = process_volume(df)
volume_feed = pd.DataFrame(block_records)

In [ ]:
volume_feed.head(25)

In [ ]:
fig = px.bar(volume_feed, 
                 x=volume_feed.timestamp,
                 y=volume_feed.total_volume,
                title="CRV DEX Swaps Volume Hourly (Tokens)", 

                )
fig.update_layout(autotypenumbers='convert types')
# fig = fig_append_phase_labels(fig, window_start, window_end)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_initial_liquidation'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='orange', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liquidation', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig.update_layout(
    # title=f"Aave Combined Actions: {title} (USD)",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
fig = px.bar(volume_feed, 
                 x=volume_feed.timestamp,
                 y=volume_feed.total_volume_usd,
                title="CRV DEX Swaps Volume Hourly (Tokens)", 

                )
fig.update_layout(autotypenumbers='convert types')
# fig = fig_append_phase_labels(fig, window_start, window_end)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


fig.show()

In [17]:
block_records = process_volume(df, False)
volume_feed = pd.DataFrame(block_records)

In [ ]:
fig = px.bar(volume_feed, 
                 x=volume_feed.timestamp,
                 y=volume_feed.total_volume,
                title="CRV DEX Swaps Volume Daily (Tokens)", 

                )
fig.update_layout(autotypenumbers='convert types')
# fig = fig_append_phase_labels(fig, window_start, window_end)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
            annotation_text='Avi Setup', annotation_position="top left",
            fillcolor='orange', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_initial_liquidation'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

# fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
#             annotation_text='Liq', annotation_position="top left",
#             fillcolor='purple', opacity=0.25, line_width=0)


fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
            annotation_text='M Setup', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)
            
fig.update_layout(
    # title=f"Aave Combined Actions: {title} (USD)",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
len( [
    '0xdfd3bd446f1b7fd96dc995126ee845af0b1254cd',
    '0x80d4230c0a68fc59cb264329d3a717fcaa472a13', # Biggest Liquidator
    '0x7719494eb8f3ca261f5c806d754853dc5ce2edf7',
    '0xd542aa8f1789edf123ad816c1b59ed9fed15c50e',
    '0x9bae78d1c67826cde91b20b49690589ed0879fc7',
    '0xc37704a457b1ee87eb657cae584a34961e86acac',
    '0x1df8ea15bb725e110118f031e8e71b91abaa2a06',
    '0x8bc110db7029197c3621bea8092ab1996d5dd7be',
    '0xd911560979b78821d7b045c79e36e9cbfc2f6c6f',
    '0x45cb6131d548344c7f150d958026fe0923ea86e4',
    '0xcda3d75a1a247bf3fa9efd0727db54d7cf0c90c2',
    '0x058b10cbe1872ad139b00326686ee8ccef274c58',
    '0x6c6b87d44d239b3750bf9badce26a9a0a3d2364e',
    '0xe3b9ed955bf1c8c520bb9420abac6e62bb110b29',
    '0x0fe269d6d9e04ecc659b6e3d582a7f35ce419e0f',
    '0x95ecfcc073f1d768be35839dd27724a0aed78e60',
    '0x30be05fe3ed386b8d8afb327b03f50c9d97dcb85',
    '0xabcf5d4be599f1c7f71fcbcae4643a2aa849f4c8',
    '0xc2a54f74ebbbba5ed92d6a0b5dcb0b0ffb96f36e',
    '0x0000000000bb00d4f9ace884c5709edfcf587e1c',
    '0x4f381fb46dfde2bc9dcae2d881705749b1ed6e1a',
])